In [46]:
# Imports

# Visualisations
import matplotlib.pyplot as plt 
import matplotlib
import seaborn as sns
plt.style.use('fivethirtyeight')
%matplotlib inline

# Warnings
import warnings
warnings.filterwarnings('ignore')

# Data exploration
import pandas as pd

# Numerical
import numpy as np

# Spatial analysis
import geopandas as gpd   # used to read .shp file
from shapely.geometry import Point, Polygon, shape

# Regular expressions
import re

# Random
np.random.seed(11)


In [47]:
dft1 = pd.read_csv("./Data/NYPD_Complaint_Data_Current__Year_To_Date_.csv")
dft1.head()

,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,HADEVELOPT,HOUSING_PSA,...,SUSP_SEX,TRANSIT_DISTRICT,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon
0,457872064,113,QUEENS,06/30/2018,23:30:00,06/30/2018,23:40:00,COMPLETED,NaN,NaN,...,M,NaN,18-24,BLACK,F,1048111.0,188479.0,40.683786,-73.769746,"(40.68378593, -73.769745503)"
1,271939965,115,QUEENS,06/30/2018,23:20:00,06/30/2018,23:25:00,COMPLETED,NaN,NaN,...,M,NaN,18-24,ASIAN/PAC.ISL,M,1017940.0,214837.0,40.756298,-73.878397,"(40.756298483, -73.878397328)"
2,862644726,94,BROOKLYN,06/30/2018,23:09:00,06/30/2018,23:25:00,COMPLETED,NaN,NaN,...,M,NaN,45-64,ASIAN/PAC.ISL,M,997245.0,204130.0,40.726965,-73.953117,"(40.726965067, -73.953116629)"
3,354814756,41,BRONX,06/30/2018,23:00:00,06/30/2018,23:19:00,COMPLETED,NaN,NaN,...,M,NaN,UNKNOWN,UNKNOWN,E,1012522.0,239685.0,40.824519,-73.897849,"(40.824518508, -73.897849003)"
4,963900505,43,BRONX,06/30/2018,23:00:00,06/30/2018,23:29:00,COMPLETED,MONROE,911.0,...,M,NaN,18-24,WHITE HISPANIC,F,1021060.0,238980.0,40.822552,-73.867003,"(40.822552032, -73.867003392)"


| Index | Column Name | Column Description |
| --- | --- | --- |
| 0 | CMPLNT_NUM	 | Unique ID |
| 1,2 | CMPLNT_FR_DT, CMPLNT_FR_TM	 | Complaint From Date and Time |
| 3,4 | CMPLNT_TO_DT, CMPLNT_TO_TM	 | Complaint To Date and Time |
| 5 | RPT_DT | Report Date |
| 6,7 | KY_CD, OFNS_DESC | Criminal ID and Description |
| 8, 9 | PD_CD, PD_DESC | Police Department Code and Description |
| 10 | CRM_ATPT_CPTD_CD | Completed, Attempted but Fail, etc. |
| 11 | ADDR_PCT_CD |  |
| 13 | HADEVELOPT |  |
| 14 | HOUSING_PSA |  |
| 15 | STATION_NAME |  |
| 16 | SUSP_AGE_GROUP | Suspect Age Group |
| 17 | SUSP_RACE | Suspect Race |
| 18 | SUSP_SEX | Suspect Sex |
| 19 | TRANSIT_DISTRICT |  |
| 20 | VIC_AGE_GROUP	 | Victim Age Group |
| 21,22 | VIC_RACE | Victim Race |
| 23 | VIC_SEX | Victim Sex |
| 24,25 | X_COORD_CD | Latitude |
| 26,27 | Y_COORD_CD | Longitude |
| 28 | Longitude		 |  |
| 29 | PATROL_BORO | |
| 30 | Latitude |  |
| 31 | Lat_Lon | |


Restricting our analysis to last 8 years

### Dimensionality Reduction ###

- Drop columns not pertaining to our analysis.<br>
  - HADEVELOPT	NYCHA Housing Development<br>
  - LOC_OF_OCCUR_DESC Location of occurence.Since the level of granularity for this analysis is restricted to the boroughs this column could be dropped.
- Irrelevant attributes to our data mining task<br>
  - HOUSING_PSA<br>
  - PARK_NM<br>       
  - JURISDICTION_CODE<br>
  - PATROL_BORO <br>
  - STATION_NAME<br>
  - SUSP_AGE_GROUP<br>
  - SUSP_RACE<br>
  - SUSP_SEX<br>
  - TRANSIT_DISTRICT<br>
  - VIC_AGE_GROUP<br>
  - VIC_RACE<br>
  - VIC_SEX<br>

  

In [48]:
dft1=dft1.drop(columns=["HADEVELOPT","LOC_OF_OCCUR_DESC","HOUSING_PSA","PARKS_NM","JURISDICTION_CODE","PATROL_BORO","STATION_NAME","SUSP_AGE_GROUP","SUSP_RACE","SUSP_SEX","TRANSIT_DISTRICT","VIC_AGE_GROUP","VIC_RACE","VIC_SEX"])
dft1.head()

,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,JURIS_DESC,KY_CD,...,OFNS_DESC,PD_CD,PD_DESC,PREM_TYP_DESC,RPT_DT,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon
0,457872064,113,QUEENS,06/30/2018,23:30:00,06/30/2018,23:40:00,COMPLETED,N.Y. POLICE DEPT,578,...,HARRASSMENT 2,637.0,"HARASSMENT,SUBD 1,CIVILIAN",STREET,06/30/2018,1048111.0,188479.0,40.683786,-73.769746,"(40.68378593, -73.769745503)"
1,271939965,115,QUEENS,06/30/2018,23:20:00,06/30/2018,23:25:00,COMPLETED,N.Y. POLICE DEPT,578,...,HARRASSMENT 2,637.0,"HARASSMENT,SUBD 1,CIVILIAN",FAST FOOD,06/30/2018,1017940.0,214837.0,40.756298,-73.878397,"(40.756298483, -73.878397328)"
2,862644726,94,BROOKLYN,06/30/2018,23:09:00,06/30/2018,23:25:00,COMPLETED,N.Y. POLICE DEPT,343,...,OTHER OFFENSES RELATED TO THEF,478.0,"THEFT OF SERVICES, UNCLASSIFIE",TAXI (LIVERY LICENSED),06/30/2018,997245.0,204130.0,40.726965,-73.953117,"(40.726965067, -73.953116629)"
3,354814756,41,BRONX,06/30/2018,23:00:00,06/30/2018,23:19:00,COMPLETED,N.Y. POLICE DEPT,235,...,DANGEROUS DRUGS,567.0,"MARIJUANA, POSSESSION 4 & 5",PARK/PLAYGROUND,06/30/2018,1012522.0,239685.0,40.824519,-73.897849,"(40.824518508, -73.897849003)"
4,963900505,43,BRONX,06/30/2018,23:00:00,06/30/2018,23:29:00,COMPLETED,N.Y. HOUSING POLICE,344,...,ASSAULT 3 & RELATED OFFENSES,114.0,OBSTR BREATH/CIRCUL,RESIDENCE - PUBLIC HOUSING,06/30/2018,1021060.0,238980.0,40.822552,-73.867003,"(40.822552032, -73.867003392)"


### Missing value handling ###

- CMPLN_TO_DT - filled missing values with CMPLNT_FR_DT or RPT_DT means incident lasted a day
- CMPLNT_TO_TM - filled missing values with CMPLMNT_FR_TM means incident occured but did no last long - it is a kind of - -- simplification, detailed analysis of incidents' time is not point of thie kernel


In [49]:
df3=dft1[['CMPLNT_FR_DT','CMPLNT_TO_DT','RPT_DT']]
df3.tail()

,CMPLNT_FR_DT,CMPLNT_TO_DT,RPT_DT
228900,03/12/2015,12/28/2017,01/01/2018
228901,01/01/2013,01/01/2018,01/01/2018
228902,04/12/2012,01/01/2018,01/01/2018
228903,10/21/2011,01/01/2018,01/01/2018
228904,01/01/2008,01/01/2018,01/01/2018


In [50]:
#Filling out nans in complaint from date
def datef(row):
    
    if pd.isnull(row["CMPLNT_FR_DT"]):
        if pd.notnull((row["CMPLNT_TO_DT"])):
            return row["CMPLNT_TO_DT"]
        elif pd.notnull(row['RPT_DT']):
            return row['RPT_DT']
        else: 
            return row["CMPLNT_FR_DT"]
    else: 
        return row["CMPLNT_FR_DT"]
dft1["CMPLNT_FR_DT"]=dft1.apply(datef,axis=1)
dft1.count()

CMPLNT_NUM          228905
ADDR_PCT_CD         228905
BORO_NM             228755
CMPLNT_FR_DT        228905
CMPLNT_FR_TM        228905
CMPLNT_TO_DT        195321
CMPLNT_TO_TM        195450
CRM_ATPT_CPTD_CD    228905
JURIS_DESC          228905
KY_CD               228905
LAW_CAT_CD          228905
OFNS_DESC           228900
PD_CD               228757
PD_DESC             228757
PREM_TYP_DESC       227918
RPT_DT              228905
X_COORD_CD          228891
Y_COORD_CD          228891
Latitude            228891
Longitude           228891
Lat_Lon             228891
dtype: int64

- OFNS_DESC - 'OFNS_DESC' column has only 4 NaN values. Rows with those values have values in PD_DESC, so the best solution is to copy them. 
- PD_DESC - NaN values filled with OFNS_DESC values

In [51]:
# All NaNs from 'PD_DESC' series are filled with copy of 'OFNS_DESC' values
dft1['OFNS_DESC'] = np.where(dft1['OFNS_DESC'].isnull(), dft1['PD_DESC'], dft1['OFNS_DESC']) # There is pandas equivalent of np.where -> https://stackoverflow.com/questions/38579532/pandas-equivalent-of-np-where
# And vice versa
dft1['PD_DESC'] = np.where(dft1['PD_DESC'].isnull(), dft1['OFNS_DESC'], dft1['PD_DESC'])

# Sanity check
print(dft1['OFNS_DESC'].notnull().value_counts()) 
print(dft1['PD_DESC'].notnull().value_counts())

True    228905
Name: OFNS_DESC, dtype: int64
True    228905
Name: PD_DESC, dtype: int64


In [52]:
# There are 2 rows without values in localization series, both are from BRONX, fill it with location from bronx according to ADDR_PCT_CD value
dft1['Latitude'].fillna(40.821054, inplace = True)
dft1['Longitude'].fillna(-73.893848, inplace = True)

- Latitude & Longitude - there are 2 rows with NaNs in these columns, so I chceked ADDR_PCT_CD value and found that is Bronx, picked random location in Bronx (manually) for these 2 NaNs 
- X_COORD_CD & Y_COORD_CD - NaNs filled with converted Latitude and Longitude 
- Lat_Lon - filled with latitude and longitude as a string type

- PREM_TYP_DESC - fill NaNs with randomly picked values from locations with appropriate content

In [53]:
# Creat copy to calculate proportions.
prem_typ_desc_copy = dft1['PREM_TYP_DESC'].copy(deep = True)
prem_typ_desc_copy_rand = prem_typ_desc_copy.value_counts(normalize = True).sort_values(ascending = False)

# Fill PREM_TYP_DESC values NaN values with values from locations of other incidents.
dft1['PREM_TYP_DESC'] = dft1['PREM_TYP_DESC'].apply(lambda x: np.random.choice([x for x in dft1.prem_typ_desc],
                          replace = True, p = prem_typ_desc_copy_rand ) if (x == np.nan) else x).astype(str)

- Borough - It's time to fill NaN values in BORO_NM column. I think there is one easy, but work demanding way to fill them. NY Boroughs boundaries data will be needed. It could be obtained from NYC Open Data or directly from geopandas package which seems easier solution. Here we use data from NYC open data

In [54]:
# Values of BORO_NM colum.
dft1['BORO_NM'].value_counts(dropna = False)

BROOKLYN         67489
MANHATTAN        56691
BRONX            50153
QUEENS           44137
STATEN ISLAND    10285
NaN                150
Name: BORO_NM, dtype: int64

In [55]:
# Find rows with NaN values in crimes data frame.
nan_boros = dft1[dft1['BORO_NM'].isnull()]
nan_boros.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150 entries, 64 to 228133
Data columns (total 21 columns):
CMPLNT_NUM          150 non-null int64
ADDR_PCT_CD         150 non-null int64
BORO_NM             0 non-null object
CMPLNT_FR_DT        150 non-null object
CMPLNT_FR_TM        150 non-null object
CMPLNT_TO_DT        1 non-null object
CMPLNT_TO_TM        1 non-null object
CRM_ATPT_CPTD_CD    150 non-null object
JURIS_DESC          150 non-null object
KY_CD               150 non-null int64
LAW_CAT_CD          150 non-null object
OFNS_DESC           150 non-null object
PD_CD               2 non-null float64
PD_DESC             150 non-null object
PREM_TYP_DESC       150 non-null object
RPT_DT              150 non-null object
X_COORD_CD          150 non-null float64
Y_COORD_CD          150 non-null float64
Latitude            150 non-null float64
Longitude           150 non-null float64
Lat_Lon             150 non-null object
dtypes: float64(5), int64(3), object(13)
memory usage: 25

The idea of finding BORO_NM value is simple. Longitude and latitude values of incidents from crimes data frame indicates if point of interest lies within boundries of one from 5 different boroughs. to check it it will be used shapely package.

In [56]:
# Import NYC Borough boundries shape file. It is important to read in file using geopandas instead of pandas.
boros = gpd.read_file('./Data/geo_export_366953f5-d29c-43ab-9e8e-781c703ad083.shp')
boros['boro_name'] = boros['boro_name'].str.upper()
boros

,boro_name,boro_code,shape_leng,shape_area,geometry
0,MANHATTAN,1.0,361649.881587,6.366006e+08,(POLYGON ((-74.01092841268031 40.6844914725429...
1,BRONX,2.0,463464.521828,1.186615e+09,(POLYGON ((-73.89680883223774 40.7958084451597...
2,STATEN ISLAND,5.0,330432.867999,1.623921e+09,(POLYGON ((-74.05050806403247 40.5664220341608...
3,BROOKLYN,3.0,739945.437431,1.937567e+09,(POLYGON ((-73.86706149472118 40.5820879767934...
4,QUEENS,4.0,895228.960360,3.044772e+09,(POLYGON ((-73.83668274106707 40.5949466970158...


In [57]:
# Function to make points - credits for https://twitter.com/dangerscarf
def make_point(row):
    return Point(row.Longitude, row.Latitude)

# Go through every row, and make a point out of its lon and lat
points = dft1.apply(make_point, axis=1)

# New GeoDataFrame with old data and new geometry
dft1_geo = gpd.GeoDataFrame(dft1, geometry = points)

# It doesn't come with a CRS because it's a CSV, so let's change it. I didn`t dive deep into it,
# but I have read, that it`s needed to get right results.
dft1_geo.crs = {'init': 'epsg:4326'}

# dft1_geo.head()

In [58]:
# Fill NaN values in BORO_NM
def fill_boro_nan(gdf1, gdf2):
    """ 
    Function fills NaN values in BORO_NM column in geodataframe and returns gdf with filled values.
    gdf1 - geodataframe with NaN values and POINTS
    gdf2 - geodataframe with POLYGONS and borougs names
    """
    boro_list = [] # List for keeping values of BORO_NM
    for point in gdf1['geometry'][gdf1['BORO_NM'].isnull()]:   # Iterate through rows with NaN values in BORO_NM column
        for i in range(0, len(gdf2['geometry'])):   # Iterate through rows of boros data frame
            if point.within(gdf2['geometry'][i]):   # Check if incident is within boundaries of one of boroughs
                gdf1['BORO_NM'][gdf1['BORO_NM'].isnull()][i] = gdf2['boro_name'][i]   # Ascribe borough name to incident location
                boro_list.append(gdf2['boro_name'][i])   # Make a list of boroughs
      
    boro_s = pd.Series(v for v in boro_list)    # Change list to series
    gdf1['BORO_NM'][gdf1['BORO_NM'].isnull()] = boro_s.values    # Fill NaN values in BORO_NM
    return boro_s    # Will be usefull to check if fills are correct

In [59]:
#Execute
boro_s = fill_boro_nan(dft1_geo, boros)

In [60]:
# Check values
dft1_geo['BORO_NM'].value_counts(dropna = False)

BROOKLYN         67540
MANHATTAN        56707
BRONX            50204
QUEENS           44166
STATEN ISLAND    10288
Name: BORO_NM, dtype: int64

In [61]:
dft2=dft1.drop(columns=['CMPLNT_TO_DT',
       'CMPLNT_TO_TM', 'RPT_DT'])
dft2.head()

,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CRM_ATPT_CPTD_CD,JURIS_DESC,KY_CD,LAW_CAT_CD,OFNS_DESC,PD_CD,PD_DESC,PREM_TYP_DESC,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon,geometry
0,457872064,113,QUEENS,06/30/2018,23:30:00,COMPLETED,N.Y. POLICE DEPT,578,VIOLATION,HARRASSMENT 2,637.0,"HARASSMENT,SUBD 1,CIVILIAN",STREET,1048111.0,188479.0,40.683786,-73.769746,"(40.68378593, -73.769745503)",POINT (-73.769745503 40.68378593)
1,271939965,115,QUEENS,06/30/2018,23:20:00,COMPLETED,N.Y. POLICE DEPT,578,VIOLATION,HARRASSMENT 2,637.0,"HARASSMENT,SUBD 1,CIVILIAN",FAST FOOD,1017940.0,214837.0,40.756298,-73.878397,"(40.756298483, -73.878397328)",POINT (-73.87839732800001 40.756298483)
2,862644726,94,BROOKLYN,06/30/2018,23:09:00,COMPLETED,N.Y. POLICE DEPT,343,MISDEMEANOR,OTHER OFFENSES RELATED TO THEF,478.0,"THEFT OF SERVICES, UNCLASSIFIE",TAXI (LIVERY LICENSED),997245.0,204130.0,40.726965,-73.953117,"(40.726965067, -73.953116629)",POINT (-73.95311662899999 40.72696506699999)
3,354814756,41,BRONX,06/30/2018,23:00:00,COMPLETED,N.Y. POLICE DEPT,235,MISDEMEANOR,DANGEROUS DRUGS,567.0,"MARIJUANA, POSSESSION 4 & 5",PARK/PLAYGROUND,1012522.0,239685.0,40.824519,-73.897849,"(40.824518508, -73.897849003)",POINT (-73.897849003 40.824518508)
4,963900505,43,BRONX,06/30/2018,23:00:00,COMPLETED,N.Y. HOUSING POLICE,344,MISDEMEANOR,ASSAULT 3 & RELATED OFFENSES,114.0,OBSTR BREATH/CIRCUL,RESIDENCE - PUBLIC HOUSING,1021060.0,238980.0,40.822552,-73.867003,"(40.822552032, -73.867003392)",POINT (-73.867003392 40.822552032)


In [62]:
dft2.rename(columns={'CMPLNT_NUM':'ComplaintID', 'CMPLNT_FR_DT':'Date','CMPLNT_FR_TM':'Time',"KY_CD":"Offence Code",'OFNS_DESC':'Description','PD_CD':'Internal Code',
 'PD_DESC':'Internal Description',
 'CRM_ATPT_CPTD_CD':'Status',
 'LAW_CAT_CD':'Offence Level',
 'BORO_NM':'Borough',
 'ADDR_PCT_CD':'Neighborhood',
 'PREM_TYP_DESC':'Premise Description'}, inplace=True)

dft2.reset_index(inplace=True)
dft2.drop(columns='index')

,ComplaintID,Neighborhood,Borough,Date,Time,Status,JURIS_DESC,Offence Code,Offence Level,Description,Internal Code,Internal Description,Premise Description,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon,geometry
0,457872064,113,QUEENS,06/30/2018,23:30:00,COMPLETED,N.Y. POLICE DEPT,578,VIOLATION,HARRASSMENT 2,637.0,"HARASSMENT,SUBD 1,CIVILIAN",STREET,1048111.0,188479.0,40.683786,-73.769746,"(40.68378593, -73.769745503)",POINT (-73.769745503 40.68378593)
1,271939965,115,QUEENS,06/30/2018,23:20:00,COMPLETED,N.Y. POLICE DEPT,578,VIOLATION,HARRASSMENT 2,637.0,"HARASSMENT,SUBD 1,CIVILIAN",FAST FOOD,1017940.0,214837.0,40.756298,-73.878397,"(40.756298483, -73.878397328)",POINT (-73.87839732800001 40.756298483)
2,862644726,94,BROOKLYN,06/30/2018,23:09:00,COMPLETED,N.Y. POLICE DEPT,343,MISDEMEANOR,OTHER OFFENSES RELATED TO THEF,478.0,"THEFT OF SERVICES, UNCLASSIFIE",TAXI (LIVERY LICENSED),997245.0,204130.0,40.726965,-73.953117,"(40.726965067, -73.953116629)",POINT (-73.95311662899999 40.72696506699999)
3,354814756,41,BRONX,06/30/2018,23:00:00,COMPLETED,N.Y. POLICE DEPT,235,MISDEMEANOR,DANGEROUS DRUGS,567.0,"MARIJUANA, POSSESSION 4 & 5",PARK/PLAYGROUND,1012522.0,239685.0,40.824519,-73.897849,"(40.824518508, -73.897849003)",POINT (-73.897849003 40.824518508)
4,963900505,43,BRONX,06/30/2018,23:00:00,COMPLETED,N.Y. HOUSING POLICE,344,MISDEMEANOR,ASSAULT 3 & RELATED OFFENSES,114.0,OBSTR BREATH/CIRCUL,RESIDENCE - PUBLIC HOUSING,1021060.0,238980.0,40.822552,-73.867003,"(40.822552032, -73.867003392)",POINT (-73.867003392 40.822552032)
5,909411087,104,QUEENS,06/30/2018,23:00:00,COMPLETED,N.Y. POLICE DEPT,347,MISDEMEANOR,INTOXICATED & IMPAIRED DRIVING,905.0,"INTOXICATED DRIVING,ALCOHOL",STREET,1014406.0,198825.0,40.712362,-73.891225,"(40.712362272, -73.89122506)",POINT (-73.89122506 40.712362272)
6,287643706,67,BROOKLYN,06/30/2018,23:00:00,COMPLETED,N.Y. POLICE DEPT,578,VIOLATION,HARRASSMENT 2,638.0,"HARASSMENT,SUBD 3,4,5",STREET,997976.0,173170.0,40.641986,-73.950542,"(40.641985835, -73.950542308)",POINT (-73.950542308 40.641985835)
7,963520368,42,BRONX,06/30/2018,22:55:00,COMPLETED,N.Y. POLICE DEPT,109,FELONY,GRAND LARCENY,404.0,"LARCENY,GRAND FROM PERSON,PERSONAL ELECTRONIC ...",PARK/PLAYGROUND,1012860.0,244801.0,40.838559,-73.896606,"(40.838559364, -73.896605923)",POINT (-73.896605923 40.838559364)
8,662959967,44,BRONX,06/30/2018,22:55:00,COMPLETED,N.Y. POLICE DEPT,578,VIOLATION,HARRASSMENT 2,638.0,"HARASSMENT,SUBD 3,4,5",TAXI (LIVERY LICENSED),1005466.0,244057.0,40.836538,-73.923330,"(40.836538158, -73.923329916)",POINT (-73.923329916 40.836538158)
9,116423095,41,BRONX,06/30/2018,22:50:00,COMPLETED,N.Y. POLICE DEPT,235,MISDEMEANOR,DANGEROUS DRUGS,567.0,"MARIJUANA, POSSESSION 4 & 5",STREET,1014604.0,238800.0,40.822083,-73.890330,"(40.822082527, -73.890330326)",POINT (-73.890330326 40.822082527)


In [63]:
# to save some memory assign Borough, Status, Offence Level as 'category'
# To save some memory assign Borough, Status, Offence Level as 'category'
dft2['Borough']=dft2['Borough'].astype('category')
dft2['Offence Code']=dft2['Offence Code'].astype('category')
dft2['Internal Code']=dft2['Internal Code'].astype('category')
dft2['Status']=dft2['Status'].astype('category')
dft2['Offence Level']=dft2['Offence Level'].astype('category')
dft2['Neighborhood']=dft2['Neighborhood'].astype('category')

In [65]:
dft2.head()

,index,ComplaintID,Neighborhood,Borough,Date,Time,Status,JURIS_DESC,Offence Code,Offence Level,Description,Internal Code,Internal Description,Premise Description,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon,geometry
0,0,457872064,113,QUEENS,06/30/2018,23:30:00,COMPLETED,N.Y. POLICE DEPT,578,VIOLATION,HARRASSMENT 2,637.0,"HARASSMENT,SUBD 1,CIVILIAN",STREET,1048111.0,188479.0,40.683786,-73.769746,"(40.68378593, -73.769745503)",POINT (-73.769745503 40.68378593)
1,1,271939965,115,QUEENS,06/30/2018,23:20:00,COMPLETED,N.Y. POLICE DEPT,578,VIOLATION,HARRASSMENT 2,637.0,"HARASSMENT,SUBD 1,CIVILIAN",FAST FOOD,1017940.0,214837.0,40.756298,-73.878397,"(40.756298483, -73.878397328)",POINT (-73.87839732800001 40.756298483)
2,2,862644726,94,BROOKLYN,06/30/2018,23:09:00,COMPLETED,N.Y. POLICE DEPT,343,MISDEMEANOR,OTHER OFFENSES RELATED TO THEF,478.0,"THEFT OF SERVICES, UNCLASSIFIE",TAXI (LIVERY LICENSED),997245.0,204130.0,40.726965,-73.953117,"(40.726965067, -73.953116629)",POINT (-73.95311662899999 40.72696506699999)
3,3,354814756,41,BRONX,06/30/2018,23:00:00,COMPLETED,N.Y. POLICE DEPT,235,MISDEMEANOR,DANGEROUS DRUGS,567.0,"MARIJUANA, POSSESSION 4 & 5",PARK/PLAYGROUND,1012522.0,239685.0,40.824519,-73.897849,"(40.824518508, -73.897849003)",POINT (-73.897849003 40.824518508)
4,4,963900505,43,BRONX,06/30/2018,23:00:00,COMPLETED,N.Y. HOUSING POLICE,344,MISDEMEANOR,ASSAULT 3 & RELATED OFFENSES,114.0,OBSTR BREATH/CIRCUL,RESIDENCE - PUBLIC HOUSING,1021060.0,238980.0,40.822552,-73.867003,"(40.822552032, -73.867003392)",POINT (-73.867003392 40.822552032)


In [66]:
dft2.drop(columns=['index'],inplace=True)

In [68]:
dft2.head()

,ComplaintID,Neighborhood,Borough,Date,Time,Status,JURIS_DESC,Offence Code,Offence Level,Description,Internal Code,Internal Description,Premise Description,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon,geometry
0,457872064,113,QUEENS,06/30/2018,23:30:00,COMPLETED,N.Y. POLICE DEPT,578,VIOLATION,HARRASSMENT 2,637.0,"HARASSMENT,SUBD 1,CIVILIAN",STREET,1048111.0,188479.0,40.683786,-73.769746,"(40.68378593, -73.769745503)",POINT (-73.769745503 40.68378593)
1,271939965,115,QUEENS,06/30/2018,23:20:00,COMPLETED,N.Y. POLICE DEPT,578,VIOLATION,HARRASSMENT 2,637.0,"HARASSMENT,SUBD 1,CIVILIAN",FAST FOOD,1017940.0,214837.0,40.756298,-73.878397,"(40.756298483, -73.878397328)",POINT (-73.87839732800001 40.756298483)
2,862644726,94,BROOKLYN,06/30/2018,23:09:00,COMPLETED,N.Y. POLICE DEPT,343,MISDEMEANOR,OTHER OFFENSES RELATED TO THEF,478.0,"THEFT OF SERVICES, UNCLASSIFIE",TAXI (LIVERY LICENSED),997245.0,204130.0,40.726965,-73.953117,"(40.726965067, -73.953116629)",POINT (-73.95311662899999 40.72696506699999)
3,354814756,41,BRONX,06/30/2018,23:00:00,COMPLETED,N.Y. POLICE DEPT,235,MISDEMEANOR,DANGEROUS DRUGS,567.0,"MARIJUANA, POSSESSION 4 & 5",PARK/PLAYGROUND,1012522.0,239685.0,40.824519,-73.897849,"(40.824518508, -73.897849003)",POINT (-73.897849003 40.824518508)
4,963900505,43,BRONX,06/30/2018,23:00:00,COMPLETED,N.Y. HOUSING POLICE,344,MISDEMEANOR,ASSAULT 3 & RELATED OFFENSES,114.0,OBSTR BREATH/CIRCUL,RESIDENCE - PUBLIC HOUSING,1021060.0,238980.0,40.822552,-73.867003,"(40.822552032, -73.867003392)",POINT (-73.867003392 40.822552032)


In [69]:
dft2['Year']=dft2.Date.str[-4:]
dft2['Month']=dft2.Date.str[-10:-8]

In [70]:
dft2.groupby(by='Year')['ComplaintID'].count()

Year
1016         2
1017         1
1018        13
1025         1
1028         5
1948         1
1951         1
1969         1
1970         1
1971         2
1972         3
1973         3
1974         1
1977         3
1978         1
1979         1
1980         2
1982         2
1983         2
1985         1
1986         1
1989         1
1990         1
1991         5
1993         1
1994         5
1995         2
1996         1
1997         3
1998         8
1999         8
2000        14
2001        10
2002        11
2003        15
2004         8
2005        14
2006        12
2007        29
2008        33
2009        25
2010        56
2011        55
2012        74
2013        98
2014       126
2015       276
2016       544
2017      7397
2018    220025
Name: ComplaintID, dtype: int64

In [71]:
dft2=dft2[(dft2['Year']=='2018')]

In [72]:
dft2.groupby(by='Year')['ComplaintID'].count()

Year
2018    220025
Name: ComplaintID, dtype: int64

In [73]:
dft2['Datetime'] = dft2['Date'] + ' ' + dft2['Time']

In [74]:
dft2.tail()

,ComplaintID,Neighborhood,Borough,Date,Time,Status,JURIS_DESC,Offence Code,Offence Level,Description,...,Premise Description,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon,geometry,Year,Month,Datetime
228686,399428936,20,MANHATTAN,01/01/2018,00:00:00,COMPLETED,N.Y. POLICE DEPT,341,MISDEMEANOR,PETIT LARCENY,...,RESIDENCE - APT. HOUSE,988047.0,220354.0,40.771505,-73.986293,"(40.771504594, -73.986293046)",POINT (-73.986293046 40.771504594),2018,01,01/01/2018 00:00:00
228687,102069646,34,MANHATTAN,01/01/2018,00:00:00,COMPLETED,N.Y. POLICE DEPT,341,MISDEMEANOR,PETIT LARCENY,...,RESIDENCE - APT. HOUSE,1002777.0,248608.0,40.849035,-73.933035,"(40.849035371, -73.933034994)",POINT (-73.933034994 40.849035371),2018,01,01/01/2018 00:00:00
228688,704995508,120,STATEN ISLAND,01/01/2018,00:00:00,COMPLETED,N.Y. POLICE DEPT,578,VIOLATION,HARRASSMENT 2,...,RESIDENCE - APT. HOUSE,961172.0,162476.0,40.612614,-74.083122,"(40.612613627, -74.083122217)",POINT (-74.083122217 40.61261362699999),2018,01,01/01/2018 00:00:00
228689,524021051,14,MANHATTAN,01/01/2018,00:00:00,COMPLETED,N.Y. POLICE DEPT,341,MISDEMEANOR,PETIT LARCENY,...,BAR/NIGHT CLUB,988555.0,210445.0,40.744307,-73.984465,"(40.744306684, -73.984465345)",POINT (-73.984465345 40.744306684),2018,01,01/01/2018 00:00:00
228690,749162295,14,MANHATTAN,01/01/2018,00:00:00,COMPLETED,N.Y. POLICE DEPT,109,FELONY,GRAND LARCENY,...,STREET,988002.0,214967.0,40.756719,-73.986459,"(40.756718685, -73.986458517)",POINT (-73.986458517 40.756718685),2018,01,01/01/2018 00:00:00


In [75]:
dft2["Date"] =pd.to_datetime(dft2['Date'], format='%m/%d/%Y')

In [76]:
dft2.set_index('Date',inplace=True)

In [77]:
dft2.drop(columns='Datetime',inplace=True)

In [78]:
dft2['Year']=dft2['Year'].astype('category')
dft2['Month']=dft2['Month'].astype('category')

In [80]:
#changing numerical data into str and then into categorical data
#dft2['Neighborhood']=dft2["Neighborhood"].astype(int).astype(str).astype('category')
dft2['Internal Code']=dft2["Internal Code"].astype(str).astype('category')
dft2['Offence Code']=dft2["Offence Code"].astype(str).astype('category')

In [81]:
#saving to CSV
dft2.to_csv("NYPD_data_cleand_data_v1.csv")